In [2]:
import json

def handle_adver_type_nan(data):
    """Specifically handle NaN values in adver_type field"""
    for item in data:
        if item['data'].get('adver_type') is None or (
            isinstance(item['data'].get('adver_type'), float) and 
            str(item['data'].get('adver_type')).lower() == 'nan'
        ):
            item['data']['adver_type'] = 'Unknown'
    return data

def update_food_beverage_types(data):
    # List of adver_ids for Food & Beverage category
    food_beverage_ids = [24019, 24028, 24061, 24069, 24070, 24099]
    
    # List of terms to identify food & beverage related advertisers
    food_related_terms = [
        'restaurant', 'food', 'pizza', 'burger', 'chicken', 
        'beer', 'drink', 'bacardi', 'alcohol', 'liquor', 
        'wine', 'heineken', 'brewery', 'cafe', 'mcdonald', 
        'kfc', 'dominos', 'cadbury', 'bluebird'
    ]
    
    # Update the data
    for item in data:
        adver_id = item['data'].get('adver_id')
        advertiser = item['data'].get('advertiser', '').lower()
        
        # Check if advertisement is food/beverage related
        is_food_beverage = False
        
        # Check adver_id if it's a valid number
        if isinstance(adver_id, (int, float)) and not str(adver_id).lower() == 'nan':
            try:
                adver_id_num = int(adver_id)
                if adver_id_num in food_beverage_ids:
                    is_food_beverage = True
            except (ValueError, TypeError):
                pass
        
        # Check advertiser name
        if any(term in advertiser for term in food_related_terms):
            is_food_beverage = True
        
        if is_food_beverage:
            item['data']['adver_type'] = "Food & Beverage"
    
    return data

def main():
    filename = '/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/analysis_results.json'
    try:
        print("Starting data processing...")
        # Read the JSON file
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # First handle NaN values in adver_type
        print("Handling NaN values in adver_type...")
        data = handle_adver_type_nan(data)
        
        # Then update Food & Beverage types
        print("Updating Food & Beverage types...")
        updated_data = update_food_beverage_types(data)
        
        # Write back to the same file
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(updated_data, file, indent=2, ensure_ascii=False)
            
        print("Successfully updated advertisement types!")
        
        # Print summary of updates
        food_bev_count = sum(1 for item in updated_data 
                            if item['data'].get('adver_type') == "Food & Beverage")
        unknown_count = sum(1 for item in updated_data 
                          if item['data'].get('adver_type') == "Unknown")
        print(f"Number of advertisements categorized as Food & Beverage: {food_bev_count}")
        print(f"Number of advertisements categorized as Unknown: {unknown_count}")
        
    except FileNotFoundError:
        print("Error: Input file not found!")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in input file!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Starting data processing...
Handling NaN values in adver_type...
Updating Food & Beverage types...
Successfully updated advertisement types!
Number of advertisements categorized as Food & Beverage: 25
Number of advertisements categorized as Unknown: 171


In [1]:
import json

def handle_adver_type_nan(data):
    """Specifically handle NaN values in adver_type field"""
    for item in data:
        if item['data'].get('adver_type') is None or (
            isinstance(item['data'].get('adver_type'), float) and 
            str(item['data'].get('adver_type')).lower() == 'nan'
        ):
            item['data']['adver_type'] = 'Unknown'
    return data

def categorize_advertisement(advertiser, content):
    """Categorize advertisement based on advertiser name and content"""
    advertiser = advertiser.lower()
    content = content.lower() if content else ""
    
    # Define category keywords
    categories = {
        'Food & Beverage': [
            'restaurant', 'food', 'pizza', 'burger', 'chicken', 
            'beer', 'drink', 'bacardi', 'alcohol', 'liquor', 
            'wine', 'heineken', 'brewery', 'cafe', 'mcdonald', 
            'kfc', 'dominos', 'cadbury', 'bluebird'
        ],
        'Financial Services': [
            'bank', 'insurance', 'financial', 'money', 'investment',
            'payment', 'loan', 'credit', 'fund', 'amp', 'kiwibank',
            'investors'
        ],
        'Telecommunications': [
            'mobile', 'phone', 'telecom', 'broadband', 'internet',
            '2degrees', 'vodafone', 'one nz', 'spark', 'orcon'
        ],
        'Retail': [
            'shop', 'store', 'retail', 'mart', 'supermarket',
            'warehouse', 'woolworths', 'countdown', 'pak n save',
            'new world', 'briscoes'
        ],
        'Healthcare': [
            'health', 'medical', 'pharmacy', 'hospital', 'clinic',
            'dental', 'healthcare', 'medicine', 'pharmaceutical'
        ],
        'Automotive': [
            'car', 'vehicle', 'auto', 'motor', 'honda', 'toyota',
            'ford', 'mitsubishi', 'tire', 'automotive'
        ],
        'Entertainment': [
            'game', 'gaming', 'movie', 'film', 'tv', 'television',
            'streaming', 'entertainment', 'media', 'tvnz'
        ],
        'Travel & Tourism': [
            'travel', 'flight', 'airline', 'tourism', 'holiday',
            'vacation', 'hotel', 'air nz', 'airways'
        ],
        'Public Services': [
            'government', 'council', 'public', 'ministry', 'department',
            'service', 'emergency', 'police', 'fire'
        ],
        'Energy & Utilities': [
            'energy', 'power', 'electricity', 'gas', 'utility',
            'contact', 'genesis', 'mercury'
        ]
    }
    
    # Check each category's keywords
    for category, keywords in categories.items():
        if any(keyword in advertiser or keyword in content for keyword in keywords):
            return category
            
    return 'Unknown'

def update_advertisement_types(data):
    # First handle NaN values
    data = handle_adver_type_nan(data)
    
    # Update categories
    for item in data:
        advertiser = item['data'].get('advertiser', '')
        content = item['data'].get('content', '')
        
        category = categorize_advertisement(advertiser, content)
        if category != 'Unknown' or item['data']['adver_type'] == 'Unknown':
            item['data']['adver_type'] = category
    
    return data

def main():
    filename = '/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/complaint_analysis_results.json'
    try:
        print("Starting data processing...")
        # Read the JSON file
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Update advertisement types
        print("Updating advertisement types...")
        updated_data = update_advertisement_types(data)
        
        # Write back to the same file
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(updated_data, file, indent=2, ensure_ascii=False)
            
        print("Successfully updated advertisement types!")
        
        # Print summary of updates
        categories = {}
        for item in updated_data:
            category = item['data'].get('adver_type')
            categories[category] = categories.get(category, 0) + 1
            
        print("\nCategory Distribution:")
        for category, count in sorted(categories.items()):
            print(f"{category}: {count} advertisements")
        
    except FileNotFoundError:
        print("Error: Input file not found!")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in input file!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Starting data processing...
Updating advertisement types...
An error occurred: 'float' object has no attribute 'lower'


In [3]:
import json

def list_unknown_types(data):
    """List all advertisements with Unknown type"""
    unknown_ads = []
    for item in data:
        if item['data'].get('adver_type') == 'Unknown':
            unknown_ads.append({
                'complaint_number': item['data'].get('complaint_number', 'N/A'),
                'advertiser': item['data'].get('advertiser', 'N/A'),
                'advertisement': item['data'].get('advertisement', 'N/A'),
                'content': item['data'].get('content', 'N/A')[:100] + '...' # First 100 chars of content
            })
    return unknown_ads

def main():
    filename = '/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/complaint_analysis_results.json'
    try:
        print("Starting to read data...")
        # Read the JSON file
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Get unknown type advertisements
        unknown_ads = list_unknown_types(data)
        
        # Print results
        print(f"\nFound {len(unknown_ads)} advertisements with Unknown type:")
        print("\nDetails of Unknown type advertisements:")
        print("-" * 80)
        
        for ad in unknown_ads:
            print(f"Complaint Number: {ad['complaint_number']}")
        
        # Optionally save to a separate file
        output_filename = 'unknown_type_ads.json'
        with open(output_filename, 'w', encoding='utf-8') as file:
            json.dump(unknown_ads, file, indent=2, ensure_ascii=False)
        print(f"\nDetailed results have been saved to {output_filename}")
        
    except FileNotFoundError:
        print("Error: Input file not found!")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in input file!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Starting to read data...

Found 0 advertisements with Unknown type:

Details of Unknown type advertisements:
--------------------------------------------------------------------------------

Detailed results have been saved to unknown_type_ads.json


In [6]:
import json
from collections import defaultdict

def classify_industry(advertiser, advertisement):
    """根据广告主和广告名称分类行业"""
    text = f"{str(advertiser)} {str(advertisement)}".lower()
    
    # 定义行业分类规则
    industry_rules = {
        'Food and Beverage': [
            'food', 'restaurant', 'cafe', 'drink', 'beverage', 'pizza', 'burger',
            'meat', 'dairy', 'beer', 'wine', 'liquor', 'coffee', 'bakery',
            'supermarket', 'grocery', 'snack', 'fast food'
        ],
        'Financial Services': [
            'bank', 'insurance', 'loan', 'credit', 'mortgage', 'finance',
            'investment', 'trading', 'fund', 'money', 'financial', 'banking',
            'kiwisaver', 'capital', 'wealth'
        ],
        'Retail': [
            'shop', 'store', 'retail', 'mall', 'outlet', 'market',
            'warehouse', 'mart', 'shopping', 'department store', 'clothing',
            'fashion', 'apparel', 'shoes', 'jewelry'
        ],
        'Automotive': [
            'car', 'vehicle', 'auto', 'motor', 'toyota', 'ford', 'honda',
            'dealer', 'automotive', 'garage', 'repair', 'service center',
            'parts', 'tyre', 'rental'
        ],
        'Healthcare': [
            'health', 'medical', 'hospital', 'clinic', 'doctor', 'pharmacy',
            'dental', 'healthcare', 'medicine', 'treatment', 'wellness',
            'supplement', 'vitamin'
        ],
        'Technology': [
            'tech', 'computer', 'phone', 'mobile', 'software', 'app',
            'digital', 'electronic', 'device', 'gadget', 'internet',
            'telecom', 'broadband', 'gaming'
        ],
        'Real Estate': [
            'property', 'real estate', 'house', 'apartment', 'home',
            'building', 'construction', 'development', 'land', 'housing',
            'rent', 'lease'
        ],
        'Travel and Tourism': [
            'travel', 'holiday', 'vacation', 'hotel', 'motel', 'resort',
            'airline', 'flight', 'tourism', 'tour', 'booking', 'accommodation',
            'cruise', 'destination'
        ],
        'Education': [
            'school', 'university', 'college', 'education', 'training',
            'course', 'learning', 'institute', 'academic', 'study',
            'teaching', 'tutorial', 'student'
        ],
        'Entertainment': [
            'entertainment', 'movie', 'film', 'game', 'sport', 'music',
            'concert', 'event', 'show', 'theatre', 'cinema', 'streaming',
            'tv', 'television', 'media'
        ],
        'Telecommunications': [
            'telecom', 'phone', 'mobile', 'broadband', 'internet',
            'network', 'wireless', 'communication', 'data', 'fiber',
            'vodafone', 'spark', '2degrees'
        ]
    }
    
    # 检查每个行业的关键词
    for industry, keywords in industry_rules.items():
        if any(keyword in text for keyword in keywords):
            return industry
    
    return 'Others'  # 如果没有匹配任何行业

def categorize_by_industry(data):
    """对广告按行业进行分类并统计"""
    industry_categories = defaultdict(list)
    industry_counts = defaultdict(int)
    
    for item in data:
        advertiser = item['data'].get('advertiser', '')
        advertisement = item['data'].get('advertisement', '')
        industry = classify_industry(advertiser, advertisement)
        
        # 构建广告信息
        ad_info = {
            'complaint_number': item['data'].get('complaint_number', 'N/A'),
            'advertiser': advertiser,
            'advertisement': advertisement,
            'content': item['data'].get('content', 'N/A')[:100] + '...',
            'analysis_results': item['data'].get('analysis_results', [])
        }
        
        industry_categories[industry].append(ad_info)
        industry_counts[industry] += 1
    
    return industry_categories, industry_counts

def analyze_industry_issues(industry_categories):
    """分析每个行业的主要问题类型"""
    industry_analysis = {}
    
    for industry, ads in industry_categories.items():
        issue_counts = defaultdict(int)
        total_ads = len(ads)
        
        for ad in ads:
            for result in ad.get('analysis_results', []):
                category = result.get('category', '')
                if category:
                    issue_counts[category] += 1
        
        # 计算问题类型的百分比
        issue_percentages = {
            issue: (count / total_ads * 100)
            for issue, count in issue_counts.items()
        }
        
        industry_analysis[industry] = {
            'total_ads': total_ads,
            'top_issues': dict(sorted(issue_percentages.items(), 
                                    key=lambda x: x[1], 
                                    reverse=True))
        }
    
    return industry_analysis

def print_industry_examples(industry_categories):
    """打印每个行业的示例"""
    print("\n各行业广告示例:")
    print("=" * 80)
    for industry, ads in industry_categories.items():
        print(f"\n{industry} 行业示例 (共 {len(ads)} 条):")
        print("-" * 40)
        # 打印前3个示例
        for ad in ads[:3]:
            print(f"广告主: {ad['advertiser']}")
            print(f"广告: {ad['advertisement']}")
            print("-" * 20)
        if len(ads) > 3:
            print("...")

def save_industry_reports(industry_categories, industry_analysis, output_dir='industry_reports'):
    """保存行业分析报告"""
    import os
    
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存详细分类结果
    for industry, ads in industry_categories.items():
        filename = os.path.join(output_dir, f'{industry.lower().replace(" ", "_")}_ads.json')
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump({
                'ads': ads,
                'analysis': industry_analysis.get(industry, {})
            }, file, indent=2, ensure_ascii=False)
    
    # 保存总体分析报告
    summary_file = os.path.join(output_dir, 'industry_analysis_summary.json')
    with open(summary_file, 'w', encoding='utf-8') as file:
        json.dump(industry_analysis, file, indent=2, ensure_ascii=False)
    
    return output_dir

def main():
    filename = '/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/complaint_analysis_results.json'
    try:
        print("开始读取数据...")
        # 读取JSON文件
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # 按行业分类广告
        industry_categories, industry_counts = categorize_by_industry(data)
        
        # 分析行业问题
        industry_analysis = analyze_industry_issues(industry_categories)
        
        # 打印基本统计信息
        print("\n行业分布统计:")
        print("-" * 50)
        for industry, count in sorted(industry_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"{industry}: {count} 条广告")
        
        # 打印行业示例
        print_industry_examples(industry_categories)
        
        # 打印行业问题分析
        print("\n行业问题分析:")
        print("=" * 50)
        for industry, analysis in industry_analysis.items():
            print(f"\n{industry}:")
            print(f"总广告数: {analysis['total_ads']}")
            print("主要问题类型:")
            for issue, percentage in analysis['top_issues'].items():
                print(f"  - {issue}: {percentage:.1f}%")
        
        # 保存报告
        output_dir = save_industry_reports(industry_categories, industry_analysis)
        print(f"\n详细分析报告已保存到目录: {output_dir}")
        
    except FileNotFoundError:
        print("错误: 找不到输入文件!")
    except json.JSONDecodeError:
        print("错误: JSON格式无效!")
    except Exception as e:
        print(f"发生错误: {str(e)}")

if __name__ == "__main__":
    main()

开始读取数据...

行业分布统计:
--------------------------------------------------
Others: 663 条广告
Entertainment: 340 条广告
Real Estate: 226 条广告
Retail: 147 条广告
Healthcare: 135 条广告
Food and Beverage: 121 条广告
Automotive: 107 条广告
Technology: 63 条广告
Financial Services: 54 条广告
Travel and Tourism: 8 条广告
Telecommunications: 5 条广告
Education: 5 条广告

各行业广告示例:

Healthcare 行业示例 (共 135 条):
----------------------------------------
广告主: COMPLAINT ON BEHALF OF :
广告: Alcohol Healthwatch
--------------------
广告主: Rug Doctor
广告: Rug Doctor, Television
--------------------
广告主: Bio Energetic Medicine
广告: Out of Home
--------------------
...

Others 行业示例 (共 663 条):
----------------------------------------
广告主: COMPLAINANT L Twentyman
广告: Brand Developers Ltd
--------------------
广告主: NZ National Party
广告: NZ National Party, Twitter
--------------------
广告主: COMPLAIN ANT D Anton & C Clark
广告: ADVERTISER Griffins
--------------------
...

Real Estate 行业示例 (共 226 条):
----------------------------------------
广告主: Fonterra B

In [5]:
import json
from collections import defaultdict

def classify_industry(advertiser, advertisement):
    """根据广告主和广告名称分类行业"""
    text = f"{str(advertiser)} {str(advertisement)}".lower()
    
    # 特定品牌映射
    brand_mapping = {
        'Food & Beverage': {
            'companies': [
                'mcdonald', 'burger king', 'kfc', 'domino', 'pizza hut', 
                'subway', 'wendy', 'restaurant brands', 'foodstuffs',
                'countdown', 'pak n save', 'new world', 'four square',
                'sanitarium', 'goodman fielder', 'griffins', 'tip top',
                'heinz watties', 'coca-cola', 'pepsi', 'red bull',
                'nestle', 'hell pizza', 'starbucks', 'dunkin'
            ]
        },
        'Telecommunications': {
            'companies': [
                '2degrees', 'vodafone', 'spark', 'one nz', 'skinny', 
                'orcon', 'slingshot', '2 degrees', 'two degrees'
            ]
        },
        'Finance & Banking': {
            'companies': [
                'anz', 'asb', 'bnz', 'westpac', 'kiwibank', 'tsb',
                'rabobank', 'amp', 'heartland', 'co-operative bank',
                'visa', 'mastercard', 'american express'
            ]
        },
        'Retail': {
            'companies': [
                'warehouse', 'farmers', 'kmart', 'briscoes', 'rebel sport',
                'noel leeming', 'harvey norman', 'mitre 10', 'bunnings',
                'placemakers', 'jb hi-fi', 'kathmandu'
            ]
        },
        'Energy & Utilities': {
            'companies': [
                'mercury', 'meridian', 'genesis', 'contact', 'trustpower',
                'vector', 'powerco', 'flick'
            ]
        },
        'Transport & Automotive': {
            'companies': [
                'toyota', 'ford', 'holden', 'mazda', 'hyundai', 'honda',
                'air new zealand', 'jetstar', 'uber', 'ola', 'zoomy'
            ]
        },
        'Media & Entertainment': {
            'companies': [
                'sky tv', 'tvnz', 'three', 'mediaworks', 'nzme', 'stuff',
                'universal pictures', 'disney', 'netflix', 'spotify',
                'warner bros', 'paramount', 'sony pictures'
            ]
        },
        'Alcohol & Beverages': {
            'companies': [
                'db breweries', 'lion', 'heineken', 'asahi', 'carlsberg',
                'pernod ricard', 'frucor suntory', 'beam suntory',
                'independent liquor'
            ]
        }
    }
    
    # 先检查品牌匹配
    for industry, data in brand_mapping.items():
        if any(company in text for company in data['companies']):
            return industry
    
    # 如果没有匹配到具体品牌，使用关键词匹配
    industry_keywords = {
        'Government & Public Services': [
            'te whatu ora', 'health new', 'department', 'ministry', 'waka kotahi',
            'transport', 'internal affairs', 'public service', 'council', 
            'government', 'police', 'authority', 'commission'
        ],
        'Political Organizations': [
            'national party', 'act party', 'labour', 'political', 'campaign',
            'party', 'parliament', 'electoral'
        ],
        'Healthcare': [
            'health', 'medical', 'hospital', 'clinic', 'pharmacy', 'healthcare',
            'dental', 'doctor', 'treatment', 'therapeutic'
        ],
        'Non-Profit & Advocacy': [
            'voices for freedom', 'foundation', 'trust', 'society', 'association',
            'charity', 'advocacy', 'non-profit', 'community'
        ],
        'Education': [
            'university', 'school', 'college', 'institute', 'education',
            'training', 'learning', 'polytechnic'
        ]
    }
    
    # 检查关键词匹配
    for industry, keywords in industry_keywords.items():
        if any(keyword in text for keyword in keywords):
            return industry
    
    return 'Others'  # 如果没有匹配任何类别

def main():
    filename = '/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/complaint_analysis_results.json'
    try:
        print("开始读取数据...")
        # 读取JSON文件
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # 分类统计
        industry_counts = defaultdict(int)
        industry_examples = defaultdict(list)
        
        for item in data:
            advertiser = item['data'].get('advertiser', '')
            advertisement = item['data'].get('advertisement', '')
            industry = classify_industry(advertiser, advertisement)
            
            industry_counts[industry] += 1
            
            # 保存示例（每个行业最多保存5个）
            if len(industry_examples[industry]) < 5:
                industry_examples[industry].append({
                    'advertiser': advertiser,
                    'advertisement': advertisement
                })
        
        # 打印结果
        print("\n行业分布统计:")
        print("-" * 50)
        for industry, count in sorted(industry_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"{industry}: {count} 条广告")
            print("示例:")
            for example in industry_examples[industry]:
                print(f"  - {example['advertiser']} : {example['advertisement']}")
            print()
        
    except Exception as e:
        print(f"发生错误: {str(e)}")

if __name__ == "__main__":
    main()

开始读取数据...

行业分布统计:
--------------------------------------------------
Others: 1042 条广告
示例:
  - COMPLAINANT L Twentyman : Brand Developers Ltd
  - Fonterra Brands (New Zealand) : Limited
  - The Griffin's Food Company : Toffee Pops, Television
  - NatuneHeal : Digital Marketing
  - COMPLA INANT B Summers : ADVERTISER Great Moscow Circus

Government & Public Services: 161 条广告
示例:
  - ADVERTISERS Department of Conservation & : Waikato Regional Council
  - The Department of Internal Affairs : Keep it Real Online, Television
  - NZ Transport Agency : NZ Transport Agency, Television
  - Waka Kotahi NZ Transport : Agency
  - Department of Internal Affairs : Keep it Real Online, Television

Non-Profit & Advocacy: 149 条广告
示例:
  - COMPLAINANT 20/096 : Society for Science Based
  - COMPLAINANT Problem Gambling Foundation of : NZ
  - COMPLAINT ON BEHALF OF 20/166 : The Society for Science Based
  - Southland Building Society : SBS, Print
  - New Zealand Drug Foundation : New Zealand Drug Foundatio

加时间

In [8]:
import pandas as pd
import random
from datetime import datetime, timedelta
import re

def is_valid_date(date_str):
    """
    检查字符串是否为有效的日期格式，支持三种格式：
    1. DD/MM/YYYY
    2. DD/MM/YYYY HH:MM
    3. D-MMM-YY
    
    Args:
        date_str: 日期字符串
    Returns:
        bool: 是否为有效日期
    """
    if pd.isna(date_str) or str(date_str).strip() == '':
        return False
        
    date_str = str(date_str).strip()
    
    # 尝试匹配三种格式
    patterns = [
        r'^\d{2}/\d{2}/\d{4}$',                    # DD/MM/YYYY
        r'^\d{2}/\d{2}/\d{4}\s+\d{2}:\d{2}$',     # DD/MM/YYYY HH:MM
        r'^\d{1,2}-[A-Za-z]{3}-\d{2}$'            # D-MMM-YY
    ]
    
    return any(re.match(pattern, date_str) for pattern in patterns)

def parse_date(date_str):
    """
    解析不同格式的日期字符串
    
    Args:
        date_str: 日期字符串
    Returns:
        pd.Timestamp: 解析后的日期，如果无效则返回pd.NaT
    """
    if not isinstance(date_str, str) or date_str.strip() == '':
        return pd.NaT
        
    date_str = date_str.strip()
    try:
        # 尝试解析 DD/MM/YYYY 或 DD/MM/YYYY HH:MM
        if '/' in date_str:
            if ' ' in date_str:  # 带时间的格式
                return pd.to_datetime(date_str, format='%d/%m/%Y %H:%M')
            return pd.to_datetime(date_str, format='%d/%m/%Y')
        
        # 尝试解析 D-MMM-YY
        elif '-' in date_str:
            return pd.to_datetime(date_str, format='%d-%b-%y')
            
        return pd.NaT
    except:
        return pd.NaT

def fill_missing_dates(file_path, output_path):
    """
    填充CSV文件中缺失或无效的日期
    
    Args:
        file_path (str): 输入CSV文件路径
        output_path (str): 输出CSV文件路径
    """
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    print("开始处理数据...")
    total_rows = len(df)
    invalid_dates_count = 0
    
    # 首先检查并统计无效日期
    for index, date_str in enumerate(df['date_of_meeting']):
        if not is_valid_date(date_str):
            invalid_dates_count += 1
            if index < 5:  # 显示前5个无效日期示例
                print(f"发现无效日期: '{date_str}' 在第 {index + 1} 行")
    
    print(f"\n总行数: {total_rows}")
    print(f"无效日期数量: {invalid_dates_count}")
    
    # 转换日期列
    df['date_of_meeting'] = df['date_of_meeting'].apply(parse_date)
    
    # 按complaint_id排序
    df = df.sort_values('complaint_id')
    
    # 用于存储上一个有效日期和decision_id对应的日期
    last_valid_date = None
    decision_dates = {}
    
    # 遍历每一行
    processed_count = 0
    for index, row in df.iterrows():
        if pd.isna(row['date_of_meeting']):
            if row['decision_id'] in decision_dates:
                # 如果是相同的decision_id，使用之前保存的日期
                df.at[index, 'date_of_meeting'] = decision_dates[row['decision_id']]
            elif last_valid_date is not None:
                # 生成新的随机日期（原日期加1-5天）
                random_days = random.randint(1, 5)
                new_date = last_valid_date + timedelta(days=random_days)
                df.at[index, 'date_of_meeting'] = new_date
                decision_dates[row['decision_id']] = new_date
                processed_count += 1
            else:
                # 如果没有之前的有效日期，使用默认日期
                default_date = pd.to_datetime('2020-01-01')
                df.at[index, 'date_of_meeting'] = default_date
                decision_dates[row['decision_id']] = default_date
                processed_count += 1
        else:
            # 更新上一个有效日期和decision_id的日期
            last_valid_date = row['date_of_meeting']
            decision_dates[row['decision_id']] = last_valid_date
    
    # 将日期统一转换为标准格式 DD/MM/YYYY
    df['date_of_meeting'] = df['date_of_meeting'].dt.strftime('%d/%m/%Y')
    
    # 保存处理后的数据
    df.to_csv(output_path, index=False)
    
    # 打印处理结果统计
    print(f"\n处理完成！")
    print(f"处理的日期数量: {processed_count}")
    print(f"输出文件保存至: {output_path}")

# 使用示例
if __name__ == "__main__":
    input_file = "/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/combination_excel_withoutnull_labeled_forjson.csv"
    output_file = "/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/combination_excel_withoutnull_labeled_forjson.csv"
    fill_missing_dates(input_file, output_file)

开始处理数据...

总行数: 1874
无效日期数量: 582

处理完成！
处理的日期数量: 405
输出文件保存至: /Users/niwenyu/Desktop/OCR_PDF_EXTRACT/combination_excel_withoutnull_labeled_forjson.csv


In [13]:
import pandas as pd
import re

def create_brand_mapping():
    """创建详细的品牌和行业映射"""
    brand_mapping = {
        'Food & Beverage': {
            'companies': [
                'mcdonald', 'burger king', 'kfc', 'domino', 'pizza hut', 
                'subway', 'wendy', 'restaurant brands', 'foodstuffs',
                'countdown', 'pak n save', 'new world', 'four square',
                'sanitarium', 'goodman fielder', 'griffins', 'tip top',
                'heinz watties', 'coca-cola', 'pepsi', 'red bull',
                'nestle', 'hell pizza', 'starbucks', 'dunkin', 'carls jr',
                'tegel', 'inghams', 'fonterra', 'anchor', 'mainland',
                'kellogs', 'uncle bens', 'mars', 'cadbury', 'whittakers',
                'arnott', 'bluebird', 'eta', 'proper crisps', 'pams',
                'value', 'woolworths', 'supervalue', 'fresh choice',
                'farro', 'moore wilson', 'nosh', 'huckleberry', 'commonsense'
            ]
        },
        'Telecommunications': {
            'companies': [
                '2degrees', 'vodafone', 'spark', 'one nz', 'skinny', 
                'orcon', 'slingshot', '2 degrees', 'two degrees',
                'now', 'trustpower broadband', 'stuff fibre', 'bigpipe',
                'myrepublic', 'wireless nation', 'compass', 'farmside',
                'voyager', 'woosh', 'snap', 'vocus', 'enable networks',
                'chorus', 'ultra fast fibre', 'northpower fibre'
            ]
        },
        'Finance & Banking': {
            'companies': [
                'anz', 'asb', 'bnz', 'westpac', 'kiwibank', 'tsb',
                'rabobank', 'amp', 'heartland', 'co-operative bank',
                'visa', 'mastercard', 'american express', 'paypal',
                'afterpay', 'laybuy', 'zip', 'humm', 'flexigroup',
                'harmoney', 'latitude', 'trade me insurance',
                'southern cross', 'aai', 'tower insurance', 'fmg',
                'state insurance', 'ami', 'aa insurance', 'vero',
                'nib', 'partners life', 'sovereign', 'aig', 'shareies',
                'hatch', 'stake', 'jarden', 'craigs investment',
                'forsyth barr', 'mercer', 'kiwisaver', 'superlife'
            ]
        },
        'Retail': {
            'companies': [
                'warehouse', 'farmers', 'kmart', 'briscoes', 'rebel sport',
                'noel leeming', 'harvey norman', 'mitre 10', 'bunnings',
                'placemakers', 'jb hi-fi', 'kathmandu', 'macpac',
                'torpedo7', 'hunting & fishing', 'smiths city',
                'spotlight', 'stevens', 'bed bath & beyond', 'lighting direct',
                'pb tech', 'computer lounge', 'mighty ape', 'paper plus',
                'whitcoulls', 'hallensteins', 'glassons', 'ezibuy',
                'postie', 'just jeans', 'max', 'barkers', 'rodd & gunn',
                'h&m', 'zara', 'uniqlo', 'lush', 'myers', 'david jones'
            ]
        },
        'Energy & Utilities': {
            'companies': [
                'mercury', 'meridian', 'genesis', 'contact', 'trustpower',
                'vector', 'powerco', 'flick', 'electric kiwi', 'frank energy',
                'globug', 'ecotricity', 'powershop', 'nova energy',
                'energy online', 'pulse energy', 'grey power electricity',
                'wise prepaid', 'just energy', 'our energy', 'utilise',
                'watercare', 'wellington water', 'firstgas', 'rockgas'
            ]
        },
        'Transport & Automotive': {
            'companies': [
                'toyota', 'ford', 'holden', 'mazda', 'hyundai', 'honda',
                'air new zealand', 'jetstar', 'uber', 'ola', 'zoomy',
                'nissan', 'suzuki', 'kia', 'volkswagen', 'bmw', 'audi',
                'mercedes', 'mitsubishi', 'subaru', 'lexus', 'mini',
                'intercity', 'skip', 'lime', 'beam', 'neuron mobility',
                'at hop', 'snapper', 'metro', 'link', 'ritchies',
                'go bus', 'nz bus', 'mana coach services', 'skybus',
                'virgin australia', 'qantas', 'emirates', 'singapore airlines'
            ]
        },
        'Media & Entertainment': {
            'companies': [
                'sky tv', 'tvnz', 'three', 'mediaworks', 'nzme', 'stuff',
                'universal pictures', 'disney', 'netflix', 'spotify',
                'warner bros', 'paramount', 'sony pictures', 'prime',
                'bravo', 'eden', 'duke', 'choice tv', 'maori tv',
                'discovery', 'national geographic', 'espn', 'sky sport',
                'spark sport', 'neon', 'amazon prime', 'apple tv',
                'youtube', 'bauer media', 'radio nz', 'the spinoff',
                'newshub', '1news', 'herald', 'dominion post', 'press'
            ]
        },
        'Alcohol & Beverages': {
            'companies': [
                'db breweries', 'lion', 'heineken', 'asahi', 'carlsberg',
                'pernod ricard', 'frucor suntory', 'beam suntory',
                'independent liquor', 'steinlager', 'speights', 'corona',
                'tui', 'export gold', 'victoria bitter', 'mac', 
                'garage project', 'emersons', 'panhead', 'tuatara',
                'monteiths', 'stoke', 'behemoth', 'parrot dog',
                'absolut', 'baileys', 'jack daniels', 'jameson',
                '42 below', 'jim beam', 'bombay sapphire', 'gordon'
            ]
        },
        'Fashion & Beauty': {
            'companies': [
                'loreal', 'maybelline', 'revlon', 'mac cosmetics', 'estee lauder',
                'clinique', 'shiseido', 'sephora', 'mecca', 'farmers beauty',
                'ruby', 'glassons', 'cotton on', 'just jeans', 'max', 'supre',
                'jeans west', 'barkers', 'hallensteins', 'sports girl', 
                'forever new', 'zara', 'h&m', 'uniqlo', 'city chic'
            ]
        },
        'Sports & Recreation': {
            'companies': [
                'nike', 'adidas', 'puma', 'under armour', 'asics', 'rebel sport',
                'stirling sports', 'sport', 'fitness', 'gym', 'stadium', 
                'athletics', 'recreation', 'leisure', 'sports'
            ]
        },
        'Construction & Property': {
            'companies': [
                'fletcher', 'carter holt', 'winstone wallboards', 'james hardie',
                'carters', 'placemakers', 'mitre 10', 'bunnings', 'resene',
                'dulux', 'property', 'construction', 'build', 'homes', 'real estate'
            ]
        },
        'Technology & Electronics': {
            'companies': [
                'apple', 'samsung', 'microsoft', 'hp', 'dell', 'lenovo', 'acer',
                'asus', 'jb hi-fi', 'noel leeming', 'harvey norman', 'pb tech',
                'computer', 'electronics', 'digital', 'tech', 'software'
            ]
        }
    }

    # 扩展行业关键词映射
    industry_keywords = {
        'Government & Public Services': [
            'te whatu ora', 'health new', 'department', 'ministry', 'waka kotahi',
            'transport', 'internal affairs', 'public service', 'council', 
            'government', 'police', 'authority', 'commission', 'treasury',
            'corrections', 'defence force', 'fire and emergency', 'customs',
            'immigration', 'acc', 'worksafe', 'statistics', 'te papa',
            'reserve bank', 'crown', 'parliament', 'local board'
        ],
        'Political Organizations': [
            'national party', 'act party', 'labour', 'political', 'campaign',
            'party', 'parliament', 'electoral', 'greens', 'maori party',
            'nz first', 'democrats', 'conservative', 'opportunity party',
            'social credit', 'candidate', 'mp ', 'politician', 'election'
        ],
        'Healthcare': [
            'health', 'medical', 'hospital', 'clinic', 'pharmacy', 'healthcare',
            'dental', 'doctor', 'treatment', 'therapeutic', 'medicines',
            'physiotherapy', 'optometrist', 'surgic', 'dhb', 'ambulance',
            'st john', 'mental health', 'aged care', 'rest home', 'pharmacy',
            'chemist', 'wellness', 'care', 'rehabilitation', 'patient'
        ],
        'Non-Profit & Advocacy': [
            'voices for freedom', 'foundation', 'trust', 'society', 'association',
            'charity', 'advocacy', 'non-profit', 'community', 'greenpeace',
            'red cross', 'salvation army', 'oxfam', 'amnesty', 'unicef',
            'world vision', 'forest and bird', 'spca', 'plunket',
            'heart foundation', 'cancer society', 'ymca', 'ywca'
        ],
        'Education': [
            'university', 'school', 'college', 'institute', 'education',
            'training', 'learning', 'polytechnic', 'kindergarten', 'preschool',
            'primary school', 'high school', 'academy', 'te kura', 'wananga',
            'early childhood', 'education center', 'language school',
            'teaching', 'student', 'campus', 'faculty', 'tertiary'
        ],
        'Professional Services': [
            'law firm', 'accountant', 'consulting', 'recruitment', 'agency',
            'legal', 'advisor', 'architect', 'engineer', 'surveyor',
            'property manage', 'real estate', 'marketing', 'advertising',
            'design', 'software', 'it service', 'tech', 'digital'
        ],
        'Tourism & Hospitality': [
            'hotel', 'motel', 'resort', 'accommodation', 'tourism',
            'tourist', 'holiday', 'vacation', 'travel', 'backpacker',
            'hostel', 'lodge', 'camping', 'restaurant', 'cafe', 'bar',
            'pub', 'club', 'entertainment', 'attraction', 'tour'
        ],
        'Healthcare': [
            'health', 'medical', 'hospital', 'clinic', 'pharmacy', 'healthcare',
            'dental', 'doctor', 'treatment', 'therapeutic', 'medicines',
            'physiotherapy', 'optometrist', 'surgic', 'dhb', 'ambulance',
            'st john', 'mental health', 'aged care', 'rest home', 'pharmacy',
            'chemist', 'wellness', 'care', 'rehabilitation', 'patient',
            'nursing', 'medical center', 'health clinic', 'specialist',
            'practitioner', 'therapist', 'psychology', 'counselling'
        ],
        'Professional Services': [
            'law firm', 'accountant', 'consulting', 'recruitment', 'agency',
            'legal', 'advisor', 'architect', 'engineer', 'surveyor',
            'property manage', 'real estate', 'marketing', 'advertising',
            'design', 'software', 'it service', 'tech', 'digital',
            'service', 'consultant', 'provider', 'professional', 'advisory',
            'management', 'solution', 'expert', 'specialist', 'firm',
            'business service', 'commercial', 'corporate'
        ],
        'Manufacturing & Industrial': {
            'keywords': [
                'manufacturing', 'industrial', 'factory', 'production', 'processing',
                'engineering', 'machine', 'equipment', 'tools', 'fabrication',
                'steel', 'metal', 'plastic', 'chemical', 'packaging',
                'assembly', 'industrial equipment', 'parts', 'components'
            ]
        },
        'Construction & Property': {
            'keywords': [
                'construction', 'building', 'property', 'development', 'builder',
                'contractor', 'architecture', 'design', 'renovation', 'real estate',
                'housing', 'residential', 'commercial property', 'project',
                'development', 'infrastructure', 'civil', 'engineering'
            ]
        }
    }

    return brand_mapping, industry_keywords

def classify_advertiser(advertiser, brand_mapping, industry_keywords):
    """根据广告主名称判断其所属行业"""
    if pd.isna(advertiser) or str(advertiser).strip() == '':
        return 'Unknown'

    advertiser_lower = str(advertiser).lower().strip()
    
    # 处理特殊情况
    if advertiser_lower in ['ltd', 'limited', 'nz', 'new zealand', '']:
        return 'Unknown'

    # 首先尝试匹配具体品牌
    for industry, data in brand_mapping.items():
        for company in data['companies']:
            if company.lower() in advertiser_lower:
                return industry

    # 如果没有匹配到具体品牌，尝试关键词匹配
    for industry, keywords in industry_keywords.items():
        for keyword in keywords:
            if keyword.lower() in advertiser_lower:
                return industry

    # 根据更多通用规则进行分类
    if any(word in advertiser_lower for word in ['shop', 'store', 'retail', 'mart']):
        return 'Retail'
    if any(word in advertiser_lower for word in ['group', 'holdings', 'corporation']):
        return 'Professional Services'
    if any(word in advertiser_lower for word in ['service', 'provider', 'solutions']):
        return 'Professional Services'

    return 'Other'

def process_csv(input_file, output_file):
    """处理CSV文件并添加行业分类"""
    # 读取CSV文件
    df = pd.read_csv(input_file)
    
    # 获取品牌映射
    brand_mapping, industry_keywords = create_brand_mapping()
    
    # 添加行业分类
    df['adver_type'] = df['advertiser'].apply(
        lambda x: classify_advertiser(x, brand_mapping, industry_keywords)
    )
    
    # 保存结果
    df.to_csv(output_file, index=False)
    
    # 打印分类统计
    print("\n行业分类统计：")
    type_counts = df['adver_type'].value_counts()
    print(type_counts)
    
    # 打印一些示例进行验证
    print("\n随机抽样验证（每个类别显示一个示例）：")
    for industry in type_counts.index:
        sample = df[df['adver_type'] == industry].sample(n=1, random_state=42)
        print(f"\n{industry}:")
        print(f"Advertiser: {sample['advertiser'].values[0]}")

if __name__ == "__main__":
    input_file = "/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/combination_excel_withoutnull_labeled_forjson.csv"
    output_file = "/Users/niwenyu/Desktop/OCR_PDF_EXTRACT/combination_excel_withoutnull_labeled_forjson.csv"
    process_csv(input_file, output_file)


行业分类统计：
adver_type
Other                           793
Non-Profit & Advocacy           156
Government & Public Services    118
Food & Beverage                 113
Healthcare                       95
Transport & Automotive           90
Media & Entertainment            64
Finance & Banking                61
Retail                           59
Political Organizations          56
Alcohol & Beverages              53
Telecommunications               44
Energy & Utilities               37
Professional Services            31
Sports & Recreation              30
Tourism & Hospitality            29
Construction & Property          18
Technology & Electronics         11
Education                         8
Unknown                           5
Fashion & Beauty                  3
Name: count, dtype: int64

随机抽样验证（每个类别显示一个示例）：

Other:
Advertiser: Voice for Life

Non-Profit & Advocacy:
Advertiser: Voices for Freedom

Government & Public Services:
Advertiser: Te Whatu Ora /Health New

Food & Beverage:
A

# json格式